Just recreate the dataframe created in the part 1

In [40]:
from bs4 import BeautifulSoup
import requests
import os, os.path, csv
import pandas as pd
listingurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(listingurl)
soup = BeautifulSoup(response.text, "html.parser")
import re

listings = []
n=0
c=0
df = pd.DataFrame()
for rows in soup.find_all("td"):
    n=n+1
    r=n%3
   # print(rows)
   # if ("oddrow" in rows["class"]) or ("evenrow" in rows["class"]):
       # name = rows.find("div", class_="name").a.get_text()
       # hometown = rows.find_all("td")[1].get_text()
       # school = hometown[hometown.find(",")+4:]
       # city = hometown[:hometown.find(",")+4]
    if (r==1):
     position = str(rows).index('</td>')
     content=str(rows)[4:position]
    
     #print(content)
     
     df.loc[c,0]=content
     c=c+1
    if (r==2):
     position = str(rows).index('</td>')
     content=str(rows)[4:position]
     if (len(content)>50): 
       position = str(rows).index('">')
       position2 = str(rows).index('</a>')
       content=str(rows)[position+2:position2]
     #print(content)
     
     df.loc[c-1,1]=content
    if (r==0):
     position = str(rows).index('</td>')
     content=str(rows)[4:position]
     #print(content)

     if (len(content)>50): 
         position = str(rows).index('">')
         position2 = str(rows).index('</a>')
         content=str(rows)[position+2:position2]
         position3=[]
     df.loc[c-1,2]=content 

#remove extra rows    
df=df.loc[:287,]    
#remove /n
n=-1
for rows in df.loc[:,2]:
    n=n+1
    try:
       position=str(rows).index('\n')
       df.loc[n,2]=str(rows)[:position]
       
    except:
       a=1  
#rename the columns
df.rename(columns={0:'PostalCode',
                      1:    'Borough',
                       2:   'Neighborhood'}, 
                 inplace=True)
#remove not assigned
df=df[df.Borough != 'Not assigned']
#df=df[df.Neighborhood != 'Not assigned']
df = df.reset_index()
df=df.drop(['index'],axis=1)
#df2=pd.unique(df.PostalCode)

##concatenate Neighborhood
df3=pd.unique(df.PostalCode)
n=-1
for pc in df3:
    n=n+1
    if n==0:
        if len(df[df.PostalCode==df3[n]])==1:
          df2=df[df.PostalCode==df3[n]]
        else:
         
          df2=pd.concat([df2,df[df.PostalCode==df3[n]]])
    else:
         if len(df[df.PostalCode==df3[n]])==1:
                df2=pd.concat([df2,df[df.PostalCode==df3[n]]])
         else:       
            str0=''
            temp=df[df.PostalCode==df3[n]]
            temp=temp.reset_index() 
            temp= temp.drop(['index'],axis=1)
            
            for i in range(0,len(df[df.PostalCode==df3[n]])):

             if i==0:
                temp1=temp.loc[i,]
                str0=str0+temp1.Neighborhood
             else:
                temp1=temp.loc[i,]
                str0=str0+','+temp1.Neighborhood 
            transposed=pd.DataFrame(data=temp.iloc[0])
               
            df2=pd.concat([df2,transposed.T ])
            df2=df2.reset_index() 
            df2= df2.drop(['index'],axis=1)

            df2.loc[n,'Neighborhood']=str0

df2[df2.Neighborhood=="Not assigned"]


##add empty Neighborhood
df2.loc[4,'Neighborhood']="Queen's Park"

df2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [4]:
import sys
!{sys.executable} -m pip install geocoder

    100% |████████████████████████████████| 102kB 16.2MB/s a 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


I tried geocoder. It takes forever. A close look shows that the return value is always request denied. I am going to use the provided csv file to construct the dataframe.

In [18]:
import geocoder # import geocoder
postal_code='M5G'
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: 

Read CSV file

In [41]:
file_name='http://cocl.us/Geospatial_data'
df_coordinate=pd.read_csv(file_name)
df_coordinate.rename(columns={'Postal Code':'Postal_Code'}, 
                 inplace=True)
df_coordinate.head()


,Postal_Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Combine GPS coordinate to the postal code entries

In [42]:
Latitude=[]
Longitude=[]
for pc in df2.PostalCode:
    temp=df_coordinate[df_coordinate['Postal_Code']==pc]
    Latitude.append(float(temp.Latitude))
    Longitude.append(float(temp.Longitude))
df2['Latitude']=Latitude
df2['Longitude']=Longitude
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
